In [ ]:
import sys

res = "path-to-rice-dataset"  # https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset

In [ ]:
from safeds.data.image.containers import ImageList

rice_images, filenames = ImageList.from_files(res, return_filenames=True, load_percentage=0.05)

In [ ]:
import re
from safeds.data.tabular.containers import Column

labels = Column(
    "label", 
    [re.search(r"(.*)[\\/](.*)[\\/](.*)\.", filepath).group(2) for filepath in filenames]
)
labels

In [ ]:
from safeds.data.labeled.containers import ImageDataset

dataset = ImageDataset(rice_images, labels, batch_size=8, shuffle=True)

In [ ]:
len(dataset)

In [ ]:
from safeds.ml.nn.layers import Convolutional2DLayer, MaxPooling2DLayer, FlattenLayer, ForwardLayer

layers = [
    Convolutional2DLayer(16, 5, padding=2),
    MaxPooling2DLayer(3, stride=3),
    Convolutional2DLayer(32, 5, padding=2),
    MaxPooling2DLayer(3, stride=3),
    Convolutional2DLayer(64, 5, padding=2),
    MaxPooling2DLayer(2, stride=2),
    Convolutional2DLayer(128, 5, padding=2),
    MaxPooling2DLayer(2, stride=2),
    FlattenLayer(),
    ForwardLayer(1024),
    ForwardLayer(256),
    ForwardLayer(len(labels.get_distinct_values())),
]

In [ ]:
from safeds.ml.nn import NeuralNetworkClassifier
from safeds.ml.nn.converters import InputConversionImage, OutputConversionImageToColumn

cnn = NeuralNetworkClassifier(InputConversionImage(dataset.input_size), layers, OutputConversionImageToColumn())

In [ ]:
def batch_completion(batch_number: int, batch_loss: float):
    sys.stdout.write(f"\rbatch [{batch_number}]: {batch_loss}")
    sys.stdout.flush()

def epoch_completion(epoch_number: int, epoch_loss: float):
    sys.stdout.write(f"\repoch [{epoch_number}]: {epoch_loss}\n")
    sys.stdout.flush()

In [ ]:
trained_cnn = cnn.fit(dataset, epoch_size=5, learning_rate=0.05, callback_on_epoch_completion=epoch_completion, callback_on_batch_completion=batch_completion)